<pre>
Needs to be using python 2 kernal

python2 -m pip install ipykernel

python2 -m ipykernel install --usern

I added this to my .bashrc file

export SPARK_HOME=/home/andrew/Downloads/spark-2.1.0-bin-hadoop2.7
export PATH = $Spark_HOME/bin:$PATH


export PYTHONPATH=$SPARK_HOME/python/$PYTHONPATH
export PYTHONPATH=$SPARK_HOME/python/lib/py4j-0.10.4-src.zip$PYTHONPATH

Then finally

pip2 install findspark
</pre>

In [4]:
import findspark
findspark.init()
import pyspark
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("Read Some Parquet") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:

df = spark.read.parquet("/home/andrew/Documents/CAMCOS/Verizon/Verizon data/agg/oneday/feature/day=2016-11-25")
df.show()

+----------------+------------+-----------+-------+
|    subscriberid|featureValue|featureType|measure|
+----------------+------------+-----------+-------+
|3220237051000000|        dpcz|          4|    1.0|
|2294189755000000|       djud1|          4|    1.0|
|8102531227000000|       66413|          3|    1.0|
|5720868074000000|       57730|          3|    1.0|
|7931520141000000|       65746|          3|    1.0|
|4388889546000000|       dr77q|          4|    1.0|
|3684046523000000|       91750|          3|    1.0|
|8702309588000000|       80524|          3|    1.0|
|3506846389000000|       99502|          3|    1.0|
|6506852123000000|     9q5f36v|          4|    1.0|
|2777595771000000|       56320|          3|    1.0|
|5882619442000000|       dpq8r|          4|    1.0|
|3737765807000000|       99026|          3|    1.0|
|3754656122000000|       43964|          3|    1.0|
|2790677778000000|        9r94|          4|    1.0|
|7887428116000000|       70446|          3|    1.0|
|55395489980